<a href="https://colab.research.google.com/github/starirene9/DeepLearningAssignment/blob/main/%5B7%EC%A3%BC%EC%B0%A8%5D_%EC%9C%A0%ED%8A%9C%EB%B8%8C_%EC%9A%94%EC%95%BD_%EC%95%B1_%EB%A7%8C%EB%93%A4%EA%B8%B0_with_gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 설치
!pip install git+https://github.com/openai/whisper.git
!pip install pydub
!pip install pytubefix
!pip install --upgrade gradio
!pip install --upgrade google-genai

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-ghl38o9j
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-ghl38o9j
  Resolved https://github.com/openai/whisper.git to commit 517a43ecd132a2089d85f4ebc044728a71d49f6e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 56.0 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803707 sha256=fe918586dbc419ed0513e435e30ef1ea424593cd5807caf71c445d7b49a627d4
  Stored in directory: /tmp/pip-ephem-wheel-cache-kyvzisjk/wheels/1f/1d/98/9583695e6695a6ac0ad42d87511097dce5ba486647dbfecb0e
Successfully built openai-whisper
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.5/731.5 kB 9.1 MB/s eta 0:00:0

In [ ]:
from pytubefix import YouTube
from pydub import AudioSegment
import gradio as gr
from IPython.display import Audio, Video, display
import os
import whisper
from google import genai
from google.genai.types import GenerateContentConfig, HttpOptions

In [ ]:
# ────────────────────────────────────────────────
# 1. Gemini 키 가져오기
# ────────────────────────────────────────────────
GOOGLE_API_KEY = None
try:
    from google.colab import userdata
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
except Exception:
    pass

if not GOOGLE_API_KEY:
    GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

if not GOOGLE_API_KEY:
    raise EnvironmentError(
        "Gemini API 키를 찾을 수 없습니다.\n"
        "① Colab이라면:  `from google.colab import userdata; "
        "userdata.set('GOOGLE_API_KEY', '<키>')`  로 저장하거나\n"
        "② 환경 변수 `GOOGLE_API_KEY` 를 설정해 주세요."
    )

# ────────────────────────────────────────────────
# 2. 모델 생성
# ────────────────────────────────────────────────

# Gemini Client 생성
client = genai.Client(api_key=GOOGLE_API_KEY, http_options=HttpOptions(api_version="v1"))

# Whisper 모델
whisper_model = whisper.load_model("small")

# ────────────────────────────────────────────────
# 3. 기능 함수
# ────────────────────────────────────────────────

def make_subtitle_segments(result):
    """Whisper 자막 결과(result)에서 start, end, text를 추출해서 리스트로 변환."""
    subtitles = []
    for seg in result["segments"]:
        start = round(seg['start'], 2)  # 소수점 2자리까지
        end = round(seg['end'], 2)
        text = seg['text'].strip()
        subtitles.append((start, end, text))
    return subtitles

def split_audio(audio_path, out_path, start, end):
    audio = AudioSegment.from_file(audio_path)
    audio_clip = audio[int(start * 1000): int(end * 1000)]
    audio_clip.export(out_path, format="mp3")

def download_youtube(url: str):
    """유튜브 URL에서 비디오·오디오·자막 추출."""
    yt = YouTube(url)
    stream = yt.streams.filter(progressive=True, file_extension="mp4").order_by("resolution").first()
    if stream is None:
        raise ValueError("다운로드할 수 있는 영상이 없습니다!")

    # 다운로드 경로
    download_folder = "./"

    video_path = os.path.join(download_folder, "video.mp4")
    audio_path = os.path.join(download_folder, "audio.mp3")
    subtitle_path = os.path.join(download_folder, "subtitle.txt")
    temp_video_path = os.path.join(download_folder, "_temp_video.mp4")

    # 비디오 다운로드
    stream.download(output_path=download_folder, filename="_temp_video.mp4")
    os.replace(temp_video_path, video_path)

    # 오디오 추출
    audio = AudioSegment.from_file(video_path)
    audio.export(audio_path, format="mp3")

    # Whisper STT
    result = whisper_model.transcribe(audio_path)
    transcript = result["text"].strip()
    subtitle_segments = make_subtitle_segments(result)

    with open(subtitle_path, "w", encoding="utf-8") as f:
        f.write(transcript)

    return video_path, audio_path, subtitle_path, transcript, subtitle_segments


# 빠른 요약용
MAX_CHUNK_CHARS = 12000

def gemini_summary(transcript: str, max_sentences: int = 7) -> str:
    """유튜브 자막 기반 Gemini 요약 (신버전 API 사용)."""
    if not transcript:
        return "❗ 자막이 비어 있습니다."

    if len(transcript) > MAX_CHUNK_CHARS:
        half = MAX_CHUNK_CHARS // 2
        transcript = transcript[:half] + "\n…\n" + transcript[-half:]

    prompt = (
        f"당신은 유튜브 자막 요약가입니다.\n"
        f"아래 자막을 기반으로 핵심 내용을 5~{max_sentences}문장 이내로 요약해 주세요.\n"
        f"자막에 없는 내용은 추가하거나 창작하지 마세요.\n"
        f"요약 결과는 반드시 한국어로 작성하세요.\n\n"
        f"{transcript}"
    )

    try:
        response = client.models.generate_content(
            model="gemini-2.0-flash-001",
            contents=prompt,
            # config=GenerateContentConfig() 생략, or minimal 설정만
        )
        if response.text is None:
            return "❗ Gemini 응답에 요약 결과가 없습니다."

        return response.text.strip()

    except Exception as e:
        return f"🚨 Gemini API 호출 실패: {type(e).__name__}: {e}"

# ────────────────────────────────────────────────
# 4. Gradio 콜백
# ────────────────────────────────────────────────

def cb_download(url: str):
    try:
        # 버튼 비활성화 및 로딩 시작
        gr.update(interactive=False)

        video_path, audio_path, subtitle_path, transcript, subtitle_segments = download_youtube(url)

        return (
            video_path,
            transcript,
            subtitle_path,
            gr.update(value=subtitle_path, visible=True),
            video_path,
            gr.update(value=video_path, visible=True),
            subtitle_segments,
            audio_path,
        )
    except Exception as e:
        return gr.update(value=f"🚨 다운로드 실패: {type(e).__name__}: {e}", visible=True)
    finally:
        # 무조건 버튼 다시 활성화
        gr.update(interactive=True)


def cb_summary(transcript: str | None):
    if not transcript or not transcript.strip():
        return gr.update(value="❗ 먼저 1️⃣ 버튼으로 자막 추출 완료 후, 2️⃣ 버튼을 누르세요.", visible=True)
    try:
        summary = gemini_summary(transcript)
    except Exception as e:
        return gr.update(value=f"🚨 Gemini 오류: {type(e).__name__}: {e}", visible=True)
    return gr.update(value=summary, visible=True)

def cb_play_segment(evt: gr.SelectData, segments, audio_path):
    idx = evt.index[0]
    start, end, _ = segments.iloc[idx]
    out_path = "temp_clip.mp3"
    split_audio(audio_path, out_path, start, end)
    return gr.update(value=out_path, autoplay=True)

# ────────────────────────────────────────────────
# 5. UI 구성
# ────────────────────────────────────────────────

with gr.Blocks() as demo:
    gr.Markdown("# 🎥 YouTube 동영상 다운로드 · 자막 추출 · Gemini 요약")
    gr.Markdown(
        """
        ## 사용 방법 안내 ✨

        - 기본으로 제공된 1분짜리 YouTube URL이 입력되어 있습니다.
        - 다른 URL을 입력하면 새로운 동영상 다운로드 및 자막 추출이 가능합니다.
        - **1️⃣ 다운로드 & 자막 추출** 버튼을 누르면
            - 동영상 재생 화면
            - 자막 원문 텍스트
            - 시간별로 분리된 자막 테이블 (클릭 시 해당 구간 오디오 재생)
            - 자막 파일 미리보기, 비디오 파일 미리보기가 생성됩니다.
        - 다운로드가 완료되면 **자막 다운로드** 버튼과 **비디오 다운로드** 버튼이 활성화됩니다.
        - **2️⃣ Gemini로 요약하기** 버튼을 누르면:
            - `gemini-2.0-flash-001` 모델을 사용하여 자막을 간결하게 요약해줍니다.
        """
    )

    url_in = gr.Textbox(label="YouTube URL", value="https://www.youtube.com/watch?v=TAx_QgQmAIw")
    btn_dl = gr.Button("1️⃣ 다운로드 & 자막 추출")

    with gr.Row():
        video_out = gr.Video(label="동영상", height=460)
        subtitle_out = gr.Textbox(label="자막 (원문)", lines=20, interactive=False, value="")

    with gr.Row():
        subtitle_table = gr.Dataframe(headers=["Start (sec)", "End (sec)", "Text"], datatype=["number", "number", "str"])
        audio_player = gr.Audio(label="▶️ 선택 구간 자막 재생", visible=False)

    with gr.Row():
        subtitle_file = gr.File(label="자막 보기 (파일 미리보기)", value=None)
        video_file = gr.File(label="비디오 보기 (파일 미리보기)", value=None)

    with gr.Row():
        subtitle_download = gr.DownloadButton(label="자막 다운로드", value=None, visible=False)
        video_download = gr.DownloadButton(label="비디오 다운로드", value=None, visible=False)

    btn_sum = gr.Button("2️⃣ Gemini로 요약하기")
    summary_out = gr.Textbox(label="요약", lines=6)

    audio_path_state = gr.State()

    # 1️⃣ 버튼 연결
    btn_dl.click(
        cb_download,
        inputs=[url_in],
        outputs=[video_out, subtitle_out, subtitle_file, subtitle_download, video_file, video_download, subtitle_table, audio_path_state],
    )

    # 2️⃣ 버튼 연결 – subtitle_out (텍스트박스 값) 바로 사용
    btn_sum.click(
        cb_summary,
        inputs=[subtitle_out],
        outputs=[summary_out],
    )

    subtitle_table.select(
        fn=cb_play_segment,
        inputs=[subtitle_table, audio_path_state],
        outputs=[audio_player]
    )

# ────────────────────────────────────────────────
# 6. 실행
# ────────────────────────────────────────────────

if __name__ == "__main__":
    demo.launch(debug=True)

100%|███████████████████████████████████████| 461M/461M [00:07<00:00, 65.0MiB/s]


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://6df678cb4f59c78517.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
